<a href="https://colab.research.google.com/github/andersonfurtado/AI4WEBDEV/blob/main/ch9/Movie_Recommendation_Chatbot_multfuntional_with_Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install python-telegram-bot scikit-surprise pandas nest_asyncio textblob sqlalchemy

# Import the required libraries
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import urllib.request
import zipfile
import os
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio
from textblob import TextBlob
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker, declarative_base



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.6/631.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.1 MB/s eta 0:00:00


In [18]:
# Apply the patch to allow the use of asyncio in Jupyter Notebook
nest_asyncio.apply()

# Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# Configure the SQLite database
engine = create_engine('sqlite:///movies.db')
Base = declarative_base()

# Define the ORM classes
class Movie(Base):
    __tablename__ = 'movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'
    userId = Column(Integer, primary_key=True, autoincrement=True)
    movieId = Column(Integer)
    rating = Column(Float)
    timestamp = Column(Integer)

# Create the tables in the database
Base.metadata.create_all(engine)

# Configure the session
Session = sessionmaker(bind=engine)
session = Session()

# Insert the data into the database
movies.to_sql('movies', engine, if_exists='replace', index=False)
ratings.to_sql('ratings', engine, if_exists='replace', index=False)

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
algo = SVD()
algo.fit(trainset)

# Function to get movie recommendations directly using the database
def get_movie_recommendations_db(algo, movie_id, num_recommendations=5):
    users_who_rated_movie = session.query(Rating.userId).filter(Rating.movieId == movie_id).all()
    user_ids = [user.userId for user in users_who_rated_movie]
    other_movie_ids = session.query(Rating.movieId).filter(Rating.userId.in_(user_ids), Rating.movieId != movie_id).distinct().all()
    other_movie_ids = [movie.movieId for movie in other_movie_ids]

    predicted_ratings = []
    for other_id in other_movie_ids:
        predicted_rating = algo.predict(uid=0, iid=other_id).est
        predicted_ratings.append((other_id, predicted_rating))

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = predicted_ratings[:num_recommendations]
    recommended_movie_titles = session.query(Movie.title).filter(Movie.movieId.in_([movie_id for movie_id, _ in top_n_movies])).all()

    return [title[0] for title in recommended_movie_titles]


# Function to start the bot
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text('Hello! Send me the name of a movie and I will recommend other movies for you.')

# Function to handle text messages and recommend movies
async def handle_message(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text
    sentiment = analyze_sentiment(user_input)
    await update.message.reply_text(f"Sentiment Analysis: Polarity = {sentiment.polarity}, Subjectivity = {sentiment.subjectivity}")

    # Continue com a busca e recomendação de filmes se o sentimento for maioritariamente positivo
    if sentiment.polarity > 0:
        movie_title = user_input
        found_movies = check_movie_title(movie_title, session)
        if found_movies:
            movie_id = found_movies[0].movieId  # Assume que deseja usar a primeira correspondência
            recommendations = get_movie_recommendations_db(algo, movie_id)
            if recommendations:
                response = f"Recommendations for '{movie_title}':\n" + "\n".join(recommendations)
            else:
                response = f"No recommendations found for '{movie_title}'."
        else:
            response = "No matching movies found."
        await update.message.reply_text(response)


# Function to check if a movie title exists
def check_movie_title(title, session):
    result = session.query(Movie).filter(Movie.title.ilike(f"%{title}%")).all()
    return result

# Function for sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment


In [19]:
# Modify the test_execution_cli to use the new get_movie_recommendations_db function
def test_execution_cli():
    print("How are you feeling today?")
    user_input = input("Enter your feeling: ")
    sentiment = analyze_sentiment(user_input)
    print(f"Sentiment Analysis: Polarity = {sentiment.polarity}, Subjectivity = {sentiment.subjectivity}")

    # Prossiga apenas se o sentimento for positivo
    if sentiment.polarity > 0:
        print("Tell me the name of a movie you like:")
        movie_title = input("Enter movie title: ")

        found_movies = check_movie_title(movie_title, session)
        if found_movies:
            while True:
                print(f"Found {len(found_movies)} movie(s):")
                for i, movie in enumerate(found_movies):
                    print(f"{i+1}. Title: {movie.title}, Genres: {movie.genres}")

                print("Enter the number of the movie you are interested in:")
                selected_index = input("Enter number: ")
                if selected_index.isdigit():
                    selected_index = int(selected_index) - 1
                    if 0 <= selected_index < len(found_movies):
                        selected_movie_id = found_movies[selected_index].movieId

                        recommendations = get_movie_recommendations_db(algo, selected_movie_id)
                        if recommendations:
                            print(f"Recommendations for Movie ID '{selected_movie_id}':")
                            for i, rec in enumerate(recommendations):
                                print(f"{i+1}. {rec}")
                        else:
                            print(f"No recommendations found for Movie ID '{selected_movie_id}'.")
                        break
                    else:
                        print("Invalid selection. Please enter a valid number.")
                else:
                    print("Invalid input. Please enter a valid number.")
        else:
            print(f"No movies found with title '{movie_title}'.")
    else:
        print("Your sentiment seems a bit off, let's talk about something cheerful!")


# Run the test function for CLI
if __name__ == '__main__':
    test_execution_cli()

    # Uncomment the following line to run the bot
    # main()

How are you feeling today?
Enter your feeling: I'm fine!
Sentiment Analysis: Polarity = 0.5208333333333334, Subjectivity = 0.5
Tell me the name of a movie you like:
Enter movie title: JESUS
Found 7 movie(s):
1. Title: Jesus' Son (1999), Genres: Drama
2. Title: Jesus of Montreal (Jésus de Montréal) (1989), Genres: Drama
3. Title: Jesus Christ Superstar (1973), Genres: Drama|Musical
4. Title: Jesus Christ Vampire Hunter (2001), Genres: Action|Comedy|Horror|Musical
5. Title: Sarah Silverman: Jesus Is Magic (2005), Genres: Comedy|Musical
6. Title: Jesus Camp (2006), Genres: Documentary|Drama
7. Title: Da Sweet Blood of Jesus (2014), Genres: Comedy|Romance|Thriller
Enter the number of the movie you are interested in:
Enter number: 3
Recommendations for Movie ID '7060':
1. Shawshank Redemption, The (1994)
2. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
3. Casablanca (1942)
4. Princess Bride, The (1987)
5. Lawrence of Arabia (1962)


In [ ]:
# Token for your bot (replace with your token)
TOKEN = '7282222179:AAHTS7Ye_0WZXhzgxOVvp-jIKAVZ__fhwRA'

# Main function to run the bot
def main():
    application = Application.builder().token(TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.run_polling()

if __name__ == '__main__':
    main()